## OpenLane Colab

This Google Colab notebook will:
* Install OpenLane and its dependencies
* Run a simple design, namely a serial-parallel multiplier, through the flow
  and targeting the [open source sky130 PDK](https://github.com/google/skywater-pdk/)
  by Google and Skywater.

In [1]:
# @title Install Nix {display-mode: "form"}
# @markdown <img src="https://raw.githubusercontent.com/NixOS/nixos-artwork/master/logo/nix-snowflake.svg" width="32"/>
# @markdown
# @markdown Nix is a package manager with an emphasis on reproducible builds,
# @markdown and it is the primary method for installing OpenLane 2.
import os

%env LOCALE_ARCHIVE=/usr/lib/locale/locale-archive
!sh <(curl -L https://nixos.org/nix/install) --daemon --yes
%env PATH=/nix/var/nix/profiles/default/bin/:{os.getenv("PATH")}
!nix-env -iA cachix -f https://cachix.org/api/v1/install
!USER=root cachix use openlane

env: LOCALE_ARCHIVE=/usr/lib/locale/locale-archive
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4052  100  4052    0     0   7784      0 --:--:-- --:--:-- --:--:--  304k
downloading Nix 2.18.1 binary tarball for x86_64-linux from 'https://releases.nixos.org/nix/nix-2.18.1/nix-2.18.1-x86_64-linux.tar.xz' to '/tmp/nix-binary-tarball-unpack.dvh6XuaZhk'...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.6M  100 20.6M    0     0  66.6M      0 --:--:-- --:--:-- --:--:-- 66.8M
Note: a multi-user installation is possible. See https://nixos.org/manual/nix/stable/installation/installing-binary.html#multi-user-installation
Switching to the Multi-user Installer
Welcome to the Multi-User Ni

In [2]:
# @title Get OpenLane {display-mode: "form"}
# @markdown Click the ▷ button to download and install OpenLane.
# @markdown
# @markdown This will install OpenLane's tool dependencies using Nix,
# @markdown and OpenLane itself using PIP.

openlane_version = "latest"  # @param {key:"OpenLane Version", type:"string"}

if openlane_version == "latest":
    openlane_version = "main"

!curl -L "https://github.com/efabless/openlane2/tarball/{main}" | tar -xzC . --strip-components 1
tool_derivations = [
    "klayout.nix",
    "magic.nix",
    "netgen.nix",
    "openroad.nix",
    "yosys.nix",
]
for derivation in tool_derivations:
    !nix-env -i -f ./nix/{derivation}
!pip3 install -r ./requirements.txt
!volare enable $(cat ./openlane/open_pdks_rev)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 9415k    0 9415k    0     0  11.2M      0 --:--:-- --:--:-- --:--:-- 11.2M
installing 'klayout-6a36bfa7c04f55bd732f8e0f91b553c8f9cebed7'
these 205 paths will be fetched (198.69 MiB download, 850.18 MiB unpacked):
  /nix/store/305gzzac3s0gign6cg6fb43z94am0s17-acl-2.3.1
  /nix/store/0lnrh5y7ybsz279sn0h9ik5fmgzwzmqz-alsa-lib-1.2.7.2
  /nix/store/y93jis2i43jc86gzw7750a2zg71y2yv8-alsa-topology-conf-1.2.5.1
  /nix/store/lj28cyzhd7wc03dlk8mab9ca5sjjnw43-alsa-ucm-conf-1.2.7.1
  /nix/store/2vx8agvw4600vjjiaci61mfvm0jndnl6-at-spi2-core-2.46.0
  /nix/store/2zr7ayk7b65wn7f8ccwxq814siafmawr-attr-2.5.1
  /nix/store/4q059vhyc419r96qskj6r1hw53w7gi69-audit-2.8.5
  /nix/store/f6s8yxj8ykdmsz0s2nfvr6gkz48g67c3-avahi-0.8
  /nix/store/wim4mqpn8lxhhr10p2kd070hyj152lil-bas

In [ ]:
import openlane

print(openlane.__version__)

### Creating the design

Now that OpenLane is set up, we can write a Verilog file as follows:

In [ ]:
%%writefile spm.v
module spm(clk, rst, x, y, p);
    parameter size = 32;
    input clk, rst;
    input y;
    input[size-1:0] x;
    output p;

    wire[size-1:1] pp;
    wire[size-1:0] xy;

    genvar i;

    CSADD csa0 (.clk(clk), .rst(rst), .x(x[0]&y), .y(pp[1]), .sum(p));
    generate for(i=1; i<size-1; i=i+1) begin
        CSADD csa (.clk(clk), .rst(rst), .x(x[i]&y), .y(pp[i+1]), .sum(pp[i]));
    end endgenerate
    TCMP tcmp (.clk(clk), .rst(rst), .a(x[size-1]&y), .s(pp[size-1]));

endmodule

module TCMP(clk, rst, a, s);
    input clk, rst;
    input a;
    output reg s;

    reg z;

    always @(posedge clk or posedge rst) begin
        if (rst) begin
            //Reset logic goes here.
            s <= 1'b0;
            z <= 1'b0;
        end
        else begin
            //Sequential logic goes here.
            z <= a | z;
            s <= a ^ z;
        end
    end
endmodule

module CSADD(clk, rst, x, y, sum);
    input clk, rst;
    input x, y;
    output reg sum;

    reg sc;

    // Half Adders logic
    wire hsum1, hco1;
    assign hsum1 = y ^ sc;
    assign hco1 = y & sc;

    wire hsum2, hco2;
    assign hsum2 = x ^ hsum1;
    assign hco2 = x & hsum1;

    always @(posedge clk or posedge rst) begin
        if (rst) begin
            //Reset logic goes here.
            sum <= 1'b0;
            sc <= 1'b0;
        end
        else begin
            //Sequential logic goes here.
            sum <= hsum2;
            sc <= hco1 ^ hco2;
        end
    end
endmodule

### Setting up the configuration

OpenLane requries you to configure any Flow before using it. This is done using
the `config` module.

For colaboratories, REPLs and other interactive environments where there is no
concrete Flow object, the Configuration may be initialized using `Config.interactive`,
which will automatically propagate the configuration to any future steps.

You can find the documentation for `Config.interactive` [here](https://openlane2.readthedocs.io/en/latest/reference/api/config/index.html#openlane.config.Config.interactive).



In [ ]:
from openlane.config import Config

Config.interactive(
    "spm",
    PDK="sky130A",
    CLOCK_PORT="clk",
    CLOCK_NET="clk",
    CLOCK_PERIOD=10,
    PRIMARY_SIGNOFF_TOOL="klayout",
)

### Running implementation steps

There are two ways to obtain OpenLane's built-in implementation steps:

* via directly importing from the `steps` module using its category:
    * `from openlane.steps import Yosys` then `Synthesis = Yosys.Synthesis`
* by using the step's id from the registry:
    * `from openlane.steps import Step` then `Synthesis = Step.factory.get("Yosys.Synthesis")`

You can find a full list of included steps here: https://openlane2.readthedocs.io/en/latest/reference/step_config_vars.html

In [ ]:
from openlane.steps import Step

* First, get the step (and display its help)...

In [ ]:
Synthesis = Step.factory.get("Yosys.Synthesis")

Synthesis.display_help()

* Then run it. Note you can pass step-specific configs using Python keyword
  arguments.

In [ ]:
synthesis = Synthesis(
    VERILOG_FILES=["./spm.v"],
)
synthesis.start()

In [ ]:
display(synthesis)

In [ ]:
CheckSDCFiles = Step.factory.get("OpenROAD.CheckSDCFiles")

sdc = CheckSDCFiles()
sdc.start()

In [ ]:
Floorplan = Step.factory.get("OpenROAD.Floorplan")

floorplan = Floorplan()
floorplan.start()

In [ ]:
display(floorplan)

In [ ]:
TapEndcapInsertion = Step.factory.get("OpenROAD.TapEndcapInsertion")

tdi = TapEndcapInsertion()
tdi.start()

In [ ]:
display(tdi)

In [ ]:
IOPlacement = Step.factory.get("OpenROAD.IOPlacement")

ioplace = IOPlacement()
ioplace.start()

In [ ]:
display(ioplace)

In [ ]:
GeneratePDN = Step.factory.get("OpenROAD.GeneratePDN")

pdn = GeneratePDN()
pdn.start()

In [ ]:
display(pdn)

In [ ]:
GlobalPlacement = Step.factory.get("OpenROAD.GlobalPlacement")

gpl = GlobalPlacement()
gpl.start()

In [ ]:
display(gpl)

In [ ]:
DetailedPlacement = Step.factory.get("OpenROAD.DetailedPlacement")

dpl = DetailedPlacement()
dpl.start()

In [ ]:
display(dpl)

In [ ]:
CTS = Step.factory.get("OpenROAD.CTS")

cts = CTS()
cts.start()

In [ ]:
display(cts)

In [ ]:
GlobalRouting = Step.factory.get("OpenROAD.GlobalRouting")

grt = GlobalRouting()
grt.start()

In [ ]:
display(grt)

In [ ]:
DetailedRouting = Step.factory.get("OpenROAD.DetailedRouting")

drt = DetailedRouting()
drt.start()

In [ ]:
display(drt)

In [ ]:
FillInsertion = Step.factory.get("OpenROAD.FillInsertion")

fill = FillInsertion()
fill.start()

In [ ]:
display(fill)

In [ ]:
StreamOut = Step.factory.get("KLayout.StreamOut")

gds = StreamOut()
gds.start()

In [ ]:
display(gds)